<a href="https://colab.research.google.com/github/ryorod/meta-generative-art_dcgan/blob/master/neort_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
!pip install selenium

In [ ]:
# 参考: https://www.tomotaku.com/python-scraping-img/

In [ ]:
import os
from time import sleep
 
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
 
 
# 1枚の画像を保存する関数
def save_img(url, file_path):
    r = requests.get(url, stream=True)
 
    if r.status_code == 200:
        with open(file_path, "wb") as f:
            f.write(r.content)
 
 
# 複数の画像のダウンロードを行う関数
def download_imgs(img_urls, save_dir, tag):
    for i, url in enumerate(img_urls):
        file_name = f"{tag}_{i}.png"  # 画像ファイル名
        save_img_path = os.path.join(save_dir, file_name)  # 保存パス
 
        save_img(url, save_img_path)  # 画像の保存
 
        if (i + 1) % 10 == 0 or (i + 1) == len(img_urls):
            print(f"{i + 1} / {len(img_urls)} done")

In [ ]:
from google.colab import drive
drive.mount('./drive')

In [ ]:
save_dir = "./drive/MyDrive/neort_dataset"  # スクレイピングした画像を保存するディレクトリパス
 
# ディレクトリが存在しなければ作成する
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
# Webdriverの設定
options = Options()
options.add_argument('--headless')   # UI無しで操作する
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)  # WebDriverのパスを設定

In [ ]:
urls = []

# NEORT へアクセス
root_urls = {'p5js': "https://neort.io/tag/bmjb2ss3p9f7m1g01690", 'glsl': "https://neort.io/tag/bmj9c1c3p9fdr6916krg", 'openframeworks': "https://neort.io/tag/bmjc0643p9f7m1g01ebg", 'processing': "https://neort.io/tag/bmjb2ss3p9f7m1g0169g", 'touchdesigner': "https://neort.io/tag/bmj9a6k3p9fdr6916kp0", 'webgl': "https://neort.io/tag/bmk0qi43p9f7m1g01q1g", 'unity': "https://neort.io/tag/bmjbksc3p9f7m1g01c20", 'raymarching': "https://neort.io/tag/bmjbjcc3p9f7m1g018tg", 'dailycodingchallenge': "https://neort.io/tag/bq1sj0k3p9fefb926mp0"}

for tag in root_urls:
  prev_num_urls = len(urls)

  driver.get(root_urls[tag])  # 指定したURLへアクセス

  # 止まるまでスクロールする
  while True:
      prev_html = driver.page_source  # スクロール前のソースコード
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # 最下部までスクロール
      sleep(1.0)   # 1秒待機
      current_html = driver.page_source  # スクロール後のソースコード
 
      # スクロールの前後で変化が無ければループを抜ける
      if prev_html != current_html:
          prev_html = current_html
      else:
          break

  # 画像タグをすべて取得
  elements = driver.find_elements_by_class_name("thumb")
 
  # すべての画像URLを抜き出す
  for elem in elements:
      url = elem.get_attribute("data-src")
 
      if url not in urls and type(url) is str :
          urls.append(url)  # urlをリストに追加する
          print(url)

  urls2download = urls[prev_num_urls:]
  print(len(urls2download))
  download_imgs(urls2download, save_dir, tag)   # 画像をダウンロードする

driver.close()   # driverをクローズする